In [ ]:
import pyxatu
import pandas as pd

xatu = pyxatu.PyXatu()

color_palette = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf" 
]

In [ ]:
latest = xatu.get_slots(columns="max(slot) as slot", network="holesky")["slot"][0]
latest

In [ ]:
df = xatu.get_beacon_block_v2(slot=[latest-7200*30, latest], columns="slot, proposer_index,execution_payload_gas_limit", network="holesky")
df.sort_values("slot", inplace=True)


last_value = 30_000_000
df["execution_payload_gas_limit_adj"] = 0

for ix, i in df.iterrows():
    #print(int(i["slot"]), end="\r")
    #slot = int(i["slot"])
    #if slot != last_slot + 1:
    #    print("missing slot", slot-1)
    
    curr_value = int(i["execution_payload_gas_limit"])
    if curr_value > last_value and (last_value-curr_value) <= last_value/(1/1024):
        #print("signal       ")
        df.loc[ix, "execution_payload_gas_limit_adj"] = 60_000_000
    elif curr_value > last_value and (last_value-curr_value) > last_value/(1/1024):
        print("something strange with slot ", int(i["slot"]))
    else:
        df.loc[ix, "execution_payload_gas_limit_adj"] = 30_000_000    
    
    last_value = curr_value    
    #last_slot = slot

#df = xatu.get_beacon_block_v2(slot=[latest-7200*30, latest], columns="slot, proposer_index,execution_payload_gas_limit")
#df["execution_payload_gas_limit_adj"] = df["execution_payload_gas_limit"].apply(lambda x: 30_000_000 if x <= 30_000_000 else 60_000_000)
df["date"] = df.slot.apply(lambda x: xatu.helpers.slot_to_day(x, network="holesky"))
#df[df["execution_payload_gas_limit"] != 30000000].execution_payload_gas_limit.unique()

dfg = df.groupby(["date","execution_payload_gas_limit_adj"])["slot"].nunique().reset_index()

dfg_day = dfg.groupby("date")["slot"].sum().reset_index().rename(columns={"slot": "total"})
dfg = pd.merge(dfg, dfg_day, how="left", left_on="date", right_on="date")
dfg["signal_per"] = dfg["slot"] / dfg["total"] * 100
dfg = dfg[["date", "execution_payload_gas_limit_adj", "signal_per"]]
pivoted = dfg.pivot(index='date', columns='execution_payload_gas_limit_adj', values='signal_per')

In [ ]:
mean_gas_limit = df[df["slot"] >= df.slot.max()-7200].execution_payload_gas_limit.mean()
mean_gas_limit

In [ ]:
import plotly.graph_objects as go

def create_chart(mobile=False):

    fig = go.Figure()
    
    

    fig.add_trace(go.Bar(
        x=pivoted.index,
        y=pivoted[30000000],
        name='30m',
        marker_color=color_palette[0],
        hoverinfo='skip' 
    ))
    fig.add_trace(go.Bar(
        x=pivoted.index,
        y=pivoted[60000000],
        name='>30m',
        marker_color=color_palette[1],
        hovertemplate="%{y:.1f}%"
    ))

    

    fig.update_layout(
        title="Gas Limit Signaling (Holesky)",
        barmode='stack',
        bargap=0,
        bargroupgap=0,
        plot_bgcolor='#1c1c1c',
        paper_bgcolor='#1c1c1c',
        font=dict(family='Ubuntu Mono', size=18, color='white'),
        xaxis=dict(
            showgrid=True,
            gridcolor='gray',
            linecolor='white',
            title=None,
            tickfont=dict(size=18, color='white')
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='gray',
            linecolor='white',
            title=None,
            tickfont=dict(size=18, color='white'),
            ticksuffix='%',
            tick0=0,
            dtick=20
        ),
        legend=dict(
            bgcolor='#2b2b2b',
            bordercolor='white',
            borderwidth=1,
            font=dict(color='white')
        ),
        margin=dict(l=60, r=60, t=50, b=100),
        hovermode='x',
        hoverlabel=dict(
            font_size=20,
            font_color='white',
            bgcolor='black' 
        ),
        height=500,
        #width=1000
    )
    if mobile:
        fig.update_layout(
            margin=dict(l=60, r=60, t=150, b=100),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="center",
                x=0.5,
                bgcolor='#2b2b2b',
                bordercolor='white',
                borderwidth=1,
                font=dict(color='white')
            )
        )


    fig.update_xaxes(tickangle=45)
    return fig

In [ ]:
#fig.write_html("index.html", include_plotlyjs='inline', full_html=True)
#with open("index.html", "r", encoding="utf-8") as f:
#    html_content = f.read()
#fig.write_html("index.html", include_plotlyjs='inline', full_html=True)
#html_content

In [ ]:
desktop_fig = create_chart(mobile=False)
mobile_fig = create_chart(mobile=True)

desktop_chart_html = desktop_fig.to_html(include_plotlyjs='inline', full_html=False)
mobile_chart_html = mobile_fig.to_html(include_plotlyjs=False, full_html=False)

In [ ]:
base_html = f"""
<!doctype html>
<html lang="en">
<head>
<meta charset="UTF-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1.0"/>
<!-- Insert meta tags and CSS here -->
</head>
<body>
<!-- Header -->
<div class="header-text">
    Made with ❤️ by <a href="https://x.com/nero_eth" style="color:orange;">Toni</a> | 
    <a href="https://github.com/nerolation" style="color:orange;">GitHub</a> | Last updated: {xatu.helpers.slot_to_hour(latest, network="holesky")}<br/>
    Avg. Gas Limit (24h): {mean_gas_limit:,.0f} <br/>
    
</div>

<!-- Desktop Chart -->
<div class="desktop-chart">
    {desktop_chart_html}
</div>

<!-- Mobile Chart -->
<div class="mobile-chart">
    {mobile_chart_html}
</div>

</body>
</html>
"""

with open("index_holesky.html", "w", encoding="utf-8") as f:
    f.write(base_html)


In [ ]:
custom_css = """
<style>
body, p {
    color: #ffffff !important;
    background-color: #1c1c1c !important;
    font-family: 'Ubuntu Mono', monospace;
    margin-left: 10%;
    margin-right: 10%;
}
@media (max-width: 768px) {
    body, p {
        margin-left: 5% !important;
        margin-right: 5% !important;
    }
}
.header-text {
    text-align: center; 
    margin: 20px; 
    font-size: 16px;
    color: #cccccc; /* slightly grayer than white */
    font-family: 'Ubuntu', sans-serif; /* a rounder font */
    line-height: 1.5; /* additional line spacing */
}
.title-text {
    text-align: center;
    margin: 20px;
    font-size: 24px;
    font-weight: bold;
    color: orange;
}

/* By default, show desktop chart and hide mobile chart */
.desktop-chart { display: block; }
.mobile-chart { display: none; }

/* When screen width is small (e.g. < 768px), show mobile chart and hide desktop chart */
@media (max-width: 768px) {
    .desktop-chart { display: none; }
    .mobile-chart { display: block; }
}
</style>
"""

meta_tags = """
<meta name="description" content="A report on Gas Limit Signaling with interactive charts."/>
<meta name="keywords" content="Ethereum, Gas Limit, Blockchain, Analytics, Data Visualization"/>
<meta name="author" content="Toni Wahrstätter"/>

<!-- Open Graph Tags for social media previews -->
<meta property="og:title" content="Gas Limit Signaling Report"/>
<meta property="og:description" content="Detailed insights on Gas Limit Signaling."/>
<meta property="og:image" content="https://raw.githubusercontent.com/nerolation/gaslimit.pics/36591db90f6193f48261c0a0b035f505d378c05b/assets/previewimage.png"/>
<meta property="og:url" content="https://gaslimit.pics"/>
<meta property="og:type" content="website"/>

<!-- Twitter Card -->
<meta name="twitter:card" content="summary_large_image"/>
<meta name="twitter:title" content="Gas Limit Signaling Report"/>
<meta name="twitter:description" content="Detailed insights on Gas Limit Signaling."/>
<meta name="twitter:image" content="https://raw.githubusercontent.com/nerolation/gaslimit.pics/36591db90f6193f48261c0a0b035f505d378c05b/assets/previewimage.png"/>

<link rel="shortcut icon" href="https://mevboost.toniwahrstaetter.com/ethlogo.png">
"""

base_html = f"""
<!doctype html>
<html lang="en">
<head>
<meta charset="UTF-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1.0"/>
{meta_tags}
{custom_css}
</head>
<body>
<div class="header-text">
    Made with ❤️ by <a href="https://x.com/nero_eth" style="color:orange;">Toni</a> | 
    <a href="https://github.com/nerolation" style="color:orange;">GitHub</a> | Last updated: {xatu.helpers.slot_to_hour(latest)}<br/>
    Avg. Gas Limit (24h): {mean_gas_limit:,.0f}<br/>
    Data: <a href="https://github.com/nerolation/pyxatu" style="color:orange;">Xatu</a> by <a href="https://ethpandaops.io/" style="color:orange;">EthPandaOps</a>.
</div>

<div class="desktop-chart">
    {desktop_chart_html}
</div>

<div class="mobile-chart">
    {mobile_chart_html}
</div>
</body>
</html>
"""

with open("index_holesky.html", "w", encoding="utf-8") as f:
    f.write(base_html)
